# Prerequisites
You may have to install certain packages before running the code. If so, uncomment the line below.

In [1]:
# pip install numpy tensorflow keras nltk

# Step 1. Import Libraries
Import all the required modules. If the punkt and wordnet modules aren't already installed, uncomment the last two lines below to install them.

In [2]:
# Built in libraries
import json
import pickle
import random

# 3rd party libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Download punkt module
# nltk.download('punkt')
# nltk.download('wordnet')

# Step 2. Preprocessing the Data
Read the intents JSON data file. By observing the intents file, we can see that each tag contains a list of patterns and responses. We tokenize each pattern and add the words in a list. Also, we create a list to store tags, and a list to store documents (each document represents a pattern along with its respective intent).

In [3]:
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        # add each pattern along with its respective intent to docuemnts
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Greetings'], 'greeting'), (['Aloha'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered', '?'], 'options'), (['Help'], 'options'), (['Does', 'this', 'work', '?'], 'options'), (['What', 'is', 'this'

Now we lemmatize each word and remove the duplicate words. This way, we can reduce the number of total words in our vocabulary. At the end, we save our words and classes as files.

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
# save the python object in a file
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

70 documents
14 classes ['about', 'bank_no_offer', 'call', 'cancel', 'fees', 'goodbye', 'greeting', 'notification', 'options', 'pronounce', 'smartphone', 'thanks', 'time', 'who']
123 unique lemmatized words ["'s", 'a', 'access', 'accident', 'aloha', 'amount', 'any', 'anyone', 'app', 'are', 'awesome', 'bank', 'be', 'bye', 'call', 'can', 'cancel', 'charge', 'charged', 'chatting', 'computer', 'contact', 'cost', 'could', 'country', 'credit', 'customer', 'day', 'desk', 'do', 'doe', 'email', 'fast', 'fee', 'for', 'free', 'from', 'good', 'goodbye', 'got', 'greeting', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'internationally', 'is', 'it', 'know', 'later', 'long', 'made', 'me', 'member', 'mistake', 'money', 'much', 'my', "n't", 'need', 'next', 'nice', 'notified', 'number', 'offer', 'offered', 'on', 'online', 'or', 'other', 'payment', 'person', 'phone', 'pronounced', 'provide', 'real', 'receive', 'recieve', 'say', 'see', 'send', 'sending', 'sent', 'sm

# Step 3. Create Training and Testing Data
To train the model, we will convert each input pattern into numbers. First, we will lemmatize each word of each pattern in the documents. Then, we will create instances for training where the first feature (input) indicates which words from our vocabulary appear in the pattern, and the second feature (output) indicates which class from our list of classes the pattern belongs to.

In [5]:
# create an empty list for the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# populate training set
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern, which are then lemmatized
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # add a 1 to the bag of words if word is found in current pattern, otherwise add a 0
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a 1 for the tag of our current pattern, and 0 for any other tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    # add the input-output pair as an instance to our training set
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create input and output lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


C:\Users\iront\AppData\Local\Temp/ipykernel_1296/1204629332.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


# Step 4. Training the Model
The architecture of our model will be a neural network consisting of 3 dense layers.The dropout layers are introduced to reduce overfitting of the model. We have used the SGD optimizer and compiled the model before training. After the training of 200 epochs is completed, we then save the trained model.

In [6]:
# deep neural networds model
model = Sequential()
# add the first layer of 128 neurons that applies the rectified linear unit activation function
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# add a dropout layer to reduce overfitting. Here rate is 0.5 as the first layer has 128 neurons and the second has 64
model.add(Dropout(0.5))
# add the second layer of 64 neurons that applies the rectified linear unit activation function
model.add(Dense(64, activation='relu'))
# add another dropout layer to reduce overfitting
model.add(Dropout(0.5))
# add the final layer that has the same neurons as the number of classes.
model.add(Dense(len(train_y[0]), activation='softmax'))

# create an instance of the SGD optimizer
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# compile the model using the CategoricalCrossentropy loss function as we have 2+ label classes that are provided via one hot
# representation.
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200
14/14 [==============================] - 2s 4ms/step - loss: 2.6599 - accuracy: 0.1000
Epoch 2/200
14/14 [==============================] - 0s 4ms/step - loss: 2.5779 - accuracy: 0.1571
Epoch 3/200
14/14 [==============================] - 0s 4ms/step - loss: 2.5068 - accuracy: 0.2286
Epoch 4/200
14/14 [==============================] - 0s 5ms/step - loss: 2.3900 - accuracy: 0.3286
Epoch 5/200
14/14 [==============================] - 0s 4ms/step - loss: 2.2168 - accuracy: 0.3571
Epoch 6/200
14/14 [==============================] - 0s 5ms/step - loss: 2.1834 - accuracy: 0.3857
Epoch 7/200
14/14 [==============================] - 0s 4ms/step - loss: 2.0371 - accuracy: 0.4000
Epoch 8/200
14/14 [==============================] - 0s 4ms/step - loss: 1.9901 - accuracy: 0.4286
Epoch 9/200
14/14 [==============================] - 0s 5ms/step - loss: 1.8330 - accuracy: 0.4143
Epoch 10/200
14/14 [==============================] - 0s 4ms/step - loss: 1.6238 - accuracy: 0.4857
Epoch 11/

14/14 [==============================] - 0s 5ms/step - loss: 0.0777 - accuracy: 0.9571
Epoch 84/200
14/14 [==============================] - 0s 5ms/step - loss: 0.1098 - accuracy: 0.9714
Epoch 85/200
14/14 [==============================] - 0s 4ms/step - loss: 0.1224 - accuracy: 0.9429
Epoch 86/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0669 - accuracy: 0.9857
Epoch 87/200
14/14 [==============================] - 0s 3ms/step - loss: 0.1216 - accuracy: 0.9571
Epoch 88/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0678 - accuracy: 0.9714
Epoch 89/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0966 - accuracy: 0.9714
Epoch 90/200
14/14 [==============================] - 0s 3ms/step - loss: 0.1242 - accuracy: 0.9571
Epoch 91/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0591 - accuracy: 1.0000
Epoch 92/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0639 - accuracy: 1.0000
Epoch 93/200


14/14 [==============================] - 0s 3ms/step - loss: 0.0391 - accuracy: 0.9857
Epoch 164/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0925 - accuracy: 0.9857
Epoch 165/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0482 - accuracy: 0.9857
Epoch 166/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0247 - accuracy: 0.9857
Epoch 167/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0561 - accuracy: 0.9714
Epoch 168/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0372 - accuracy: 1.0000
Epoch 169/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0505 - accuracy: 0.9714
Epoch 170/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 171/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0448 - accuracy: 0.9857
Epoch 172/200
14/14 [==============================] - 0s 3ms/step - loss: 0.0302 - accuracy: 1.0000
Epoc